# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [11]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [12]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [13]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [14]:
newdata = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
newdata.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [15]:
table = pd.pivot_table(newdata, values='Quantity', index=['ProductName'],columns=['CustomerID'])
table = table.fillna(0)
table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [16]:
from scipy.spatial.distance import pdist, squareform
#distances = pdist(table.T, 'euclidean')
#print(squareform(distances))

distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [17]:
listaTop5 = [e for e in distances[33].sort_values(ascending=False)[1:6].index]
listaTop5

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [19]:
df = newdata.loc[listaTop5]
df.head(10)

Quantity
CustomerID ProductName                            
264        Apricots - Halves                     1
           Apricots Fresh                        1
           Bacardi Breezer - Tropical            1
           Bagel - Plain                         1
           Banana - Leaves                       1
           Bananas                               1
           Bandage - Flexible Neon               1
           Beef - Tenderlion, Center Cut         1
           Berry Brulee                          1
           Bread - Roll, Canadian Dinner         1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [20]:
newgroup = df.groupby(['ProductName']).agg({'Quantity':'sum'})
newgroup = newgroup.sort_values('Quantity', ascending = False)

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [21]:
dfmerge = newgroup.merge(table, on='ProductName')
dfmerge = dfmerge[["Quantity",33]]
dfmerge.head(5)

,Quantity,33
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Towels - Paper / Kraft,3,1.0
Soup - Campbells Bean Medley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0


In [22]:
dfzero=dfmerge[dfmerge[33]==0]
dfzero.head(5)

,Quantity,33
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Soup - Campbells Bean Medley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0
Chicken - Soup Base,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [23]:
emptyD = {}
listID = list(table)
for h in listID:
    top5 = [e for e in distances[h].sort_values(ascending=False)[1:6].index]
    df = newdata.loc[top5]
    newgroup = df.groupby(['ProductName']).agg({'Quantity':'sum'})
    newgroup = newgroup.sort_values('Quantity', ascending = False)
    dfmerge = newgroup.merge(table, on='ProductName')
    dfmerge = dfmerge[["Quantity",h]]
    dfzero=dfmerge[dfmerge[h]==0]
    dfzero=list(dfzero.index[0:5])
    emptyD[h]=dfzero

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [24]:
datafinal = pd.DataFrame(emptyD)
col = datafinal.columns
pd.melt(datafinal, value_vars=col)

,variable,value
0,33,Butter - Unsalted
1,33,Wine - Ej Gallo Sierra Valley
2,33,Soup - Campbells Bean Medley
3,33,Wine - Blue Nun Qualitatswein
4,33,Chicken - Soup Base
...,...,...
4995,98200,Beans - Wax
4996,98200,Sherry - Dry
4997,98200,Pork - Hock And Feet Attached
4998,98200,Pears - Bosc


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [25]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cityblock'))), 
                         index=table.columns, columns=table.columns)

distances
emptyD = {}
listID = list(table)
for h in listID:
    top5 = [e for e in distances[h].sort_values(ascending=False)[1:6].index]
    df = newdata.loc[top5]
    newgroup = df.groupby(['ProductName']).agg({'Quantity':'sum'})
    newgroup = newgroup.sort_values('Quantity', ascending = False)
    dfmerge = newgroup.merge(table, on='ProductName')
    dfmerge = dfmerge[["Quantity",h]]
    dfzero=dfmerge[dfmerge[h]==0]
    dfzero=list(dfzero.index[0:5])
    emptyD[h]=dfzero

In [26]:
datafinal = pd.DataFrame(emptyD)
col = datafinal.columns
pd.melt(datafinal, value_vars=col)

,variable,value
0,33,Chocolate - Dark
1,33,Wine - Charddonnay Errazuriz
2,33,Bay Leaf
3,33,"Oranges - Navel, 72"
4,33,Wine - Ej Gallo Sierra Valley
...,...,...
4995,98200,Wine - Blue Nun Qualitatswein
4996,98200,"Pepper - White, Ground"
4997,98200,Soup - Campbells Tomato Ravioli
4998,98200,Knife Plastic - White
